An analysis of data from the 2015 Nepal earthquake.\
This is an example of a dataset that is too big to open in Excel, but Python handles it in seconds.\
The Dataset is available at: https://eq2015.npc.gov.np/#/about\
The CSVs are too large for GitHub's free file limit.

In [1]:
import pandas as pd

In [2]:
#building dataframe
building_damage_df = pd.read_csv("data/csv_building_damage_assessment.csv")
building_ownership_df = pd.read_csv("data/csv_building_ownership_and_use.csv")
building_structure_df = pd.read_csv("data/csv_building_structure.csv")
building_df = building_damage_df.merge(building_ownership_df, on=['building_id', 'district_id', 'vdcmun_id', 'ward_id']).merge(building_structure_df, on=['building_id', 'district_id', 'vdcmun_id', 'ward_id'])
building_df.rename(columns={'damage_grade_x':'damage_grade','technical_solution_proposed_x':'technical_solution_proposed'}, inplace=True)
building_df.drop(columns=['damage_grade_y','technical_solution_proposed_y'], inplace=True)

#household dataframe
household_demographics_df = pd.read_csv("data/csv_household_demographics.csv")
household_earthquake_impact_df = pd.read_csv("data/csv_household_earthquake_impact.csv")
household_resources_df = pd.read_csv("data/csv_household_resources.csv")
household_df = household_demographics_df.merge(household_earthquake_impact_df, on=['household_id', 'district_id', 'vdcmun_id', 'ward_id']).merge(household_resources_df, on=['household_id', 'district_id', 'vdcmun_id', 'ward_id'])

#individual dataframe
individual_demographics_df = pd.read_csv("data/csv_individual_demographics.csv")
individual_social_security_df = pd.read_csv("data/csv_individual_social_security.csv")
individual_df = individual_demographics_df.merge(individual_social_security_df, on=['individual_id', 'district_id', 'vdcmun_id', 'ward_id'])

#mapping dataframes
mapping_df = pd.read_csv("data/mapping.csv")
# ward_mapping_df = pd.read_csv("data/ward_vdcmun_district_name_mapping.csv")

C:\Users\Gabe\AppData\Local\Temp\ipykernel_21600\2321263194.py:2: DtypeWarning: Columns (17,29,34,35,36,37,38,39,40,41,42,47,50) have mixed types. Specify dtype option on import or set low_memory=False.
  building_damage_df = pd.read_csv("data/csv_building_damage_assessment.csv")


In [3]:
print(f'the columns in building_df are: {building_df.columns.tolist()}')
print()
print(f'the columns in household_df are: {household_df.columns.tolist()}')
print()
print(f'the columns in individual_df are: {individual_df.columns.tolist()}')
print()
print(f'the columns in mapping_df are: {mapping_df.columns.tolist()}')

the columns in building_df are: ['building_id', 'district_id', 'vdcmun_id', 'ward_id', 'damage_overall_collapse', 'damage_overall_leaning', 'damage_overall_adjacent_building_risk', 'damage_foundation_severe', 'damage_foundation_moderate', 'damage_foundation_insignificant', 'damage_roof_severe', 'damage_roof_moderate', 'damage_roof_insignificant', 'damage_corner_separation_severe', 'damage_corner_separation_moderate', 'damage_corner_separation_insignificant', 'damage_diagonal_cracking_severe', 'damage_diagonal_cracking_moderate', 'damage_diagonal_cracking_insignificant', 'damage_in_plane_failure_severe', 'damage_in_plane_failure_moderate', 'damage_in_plane_failure_insignificant', 'damage_out_of_plane_failure_severe', 'damage_out_of_plane_failure_moderate', 'damage_out_of_plane_failure_insignificant', 'damage_out_of_plane_failure_walls_ncfr_severe', 'damage_out_of_plane_failure_walls_ncfr_moderate', 'damage_out_of_plane_failure_walls_ncfr_insignificant', 'damage_gable_failure_severe', 'd

In [4]:
print(f'building_df.shape = {building_df.shape}')
print(f'houshold_df.shape = {household_df.shape}')
print(f'individual_df.shape = {individual_df.shape}')

building_df.shape = (762106, 117)
houshold_df.shape = (747365, 67)
individual_df.shape = (3677133, 20)


In [5]:
num_social_security_handicap = (individual_df['has_social_security_handicap'] == 1).sum()
print(f"Number of rows with social security handicap: {num_social_security_handicap}")

Number of rows with social security handicap: 7233


In [6]:
#finding handicapped people
handicap_individual_ids = individual_df.loc[individual_df['has_social_security_handicap'] == 1, 'individual_id'].copy()
handicap_df = pd.DataFrame({'individual_id': handicap_individual_ids})
handicap_df = handicap_df.reset_index(drop=True)

In [7]:
handicap_df

,individual_id
0,12010200015101003
1,12010300112101004
2,12010600057101001
3,12020300002101006
4,12020300025101002
...,...
7228,36660600004101006
7229,36660600028101006
7230,36670100050101006
7231,36670200009101003


In [8]:
# Merge mapping_df with handicap_df
handicap_df = handicap_df.merge(mapping_df, on='individual_id')

In [9]:
print(handicap_df.head())
print(handicap_df.shape)

       individual_id    household_id   building_id
0  12010200015101003  12010200015101  120102000151
1  12010300112101004  12010300112101  120103001121
2  12010600057101001  12010600057101  120106000571
3  12020300002101006  12020300002101  120203000021
4  12020300025101002  12020300025101  120203000251
(7233, 3)


In [10]:
# list top 10 homes by number of handicapped people
most_handicapped_homes = handicap_df['household_id'].value_counts().head(10).copy()
most_handicapped_homes

20400600073101    5
28570100123101    4
36240800044101    4
21520200003101    4
12030500036101    4
24640100344101    4
12430900010101    3
30030700142101    3
30040800363101    3
36400900046101    3
Name: household_id, dtype: int64

In [11]:
# get homes with > 1 handicapped person
multi_handicapped_homes_df = handicap_df['household_id'].value_counts().copy()
multi_handicapped_homes_df = multi_handicapped_homes_df[multi_handicapped_homes_df > 1]
print(f'There are {len(multi_handicapped_homes_df)} homes with more than one handicapped person.')

There are 314 homes with more than one handicapped person.


In [12]:
multi_handicapped_homes_df

20400600073101    5
28570100123101    4
36240800044101    4
21520200003101    4
12030500036101    4
                 ..
22440700119101    2
24520200161101    2
31290400504101    2
12210800007101    2
36650400100101    2
Name: household_id, Length: 314, dtype: int64

In [13]:
unique_handicap_homes = handicap_df.household_id.unique().copy()
unique_handicap_homes_df = pd.DataFrame(unique_handicap_homes, columns=['household_id'])

print(f'There are {len(unique_handicap_homes_df)} homes with handicapped people.')

There are 6877 homes with handicapped people.


In [14]:
unique_handicap_homes_df

,household_id
0,12010200015101
1,12010300112101
2,12010600057101
3,12020300002101
4,12020300025101
...,...
6872,36660600004101
6873,36660600028101
6874,36670100050101
6875,36670200009101


In [15]:
# get buildings with a damage grade of 3, 4, or 5 and convert damage_grade to integer
damaged_buildings_df = building_df.loc[building_df['damage_grade'].isin(['Grade 3', 'Grade 4', 'Grade 5']), ['building_id', 'damage_grade']].copy()
damaged_buildings_df['damage_grade'] = damaged_buildings_df['damage_grade'].str.replace('Grade', '').astype(int)

# reset the index of the new dataframe
damaged_buildings_df.reset_index(drop=True, inplace=True)
damaged_buildings_df

,building_id,damage_grade
0,120101000011,3
1,120101000021,5
2,120101000071,5
3,120101000091,3
4,120101000101,3
...,...,...
596017,366709001241,5
596018,366709001251,5
596019,366709001261,5
596020,366709001271,5


In [16]:
# Get all individuals and households for the damaged buildings
damaged_individuals_df = damaged_buildings_df.merge(mapping_df, on='building_id')
damaged_individuals_df

,building_id,damage_grade,individual_id,household_id
0,120101000011,3,12010100001101001,12010100001101
1,120101000011,3,12010100001101002,12010100001101
2,120101000011,3,12010100001101003,12010100001101
3,120101000021,5,12010100002101001,12010100002101
4,120101000021,5,12010100002101002,12010100002101
...,...,...,...,...
2883946,366709001271,5,36670900127101001,36670900127101
2883947,366709001281,5,36670900128101001,36670900128101
2883948,366709001281,5,36670900128101002,36670900128101
2883949,366709001281,5,36670900128101003,36670900128101


In [17]:
print('There are', damaged_individuals_df['individual_id'].nunique(), "individuals in", damaged_individuals_df['building_id'].nunique(), "damaged buildings and", damaged_individuals_df['household_id'].nunique(), "damaged households.")

There are 2883951 individuals in 545440 damaged buildings and 589896 damaged households.
